In [3]:
from pyspark.sql import SparkSession
import findspark

findspark.init()
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../../config/.env')
mongo_uri = os.getenv('MONGO_URI')
client = pymongo.MongoClient(f'{mongo_uri}')
db = client['name']
collection = db['thai_name']


document = list(collection.find({},{ "_id": 0,}))
client.close()



In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame(document)
df_dummies = pd.get_dummies(df, drop_first=True)

X = df_dummies.drop('gender_m', axis=1)
y = df_dummies['gender_m']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

prediction_df = pd.DataFrame(pred, columns=['isMale'])

X_test = X_test.idxmax(axis=1).str.replace('name_','')
test = pd.concat([X_test, y_test], axis=1)
test = test.reset_index()
test = pd.concat([test, prediction_df], axis=1)
display(test)
# pred = model.predict(['rawit'])

from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)
# import pickle
# pickle.dump(model, open("model.pkl", "wb"))


,index,0,gender_m,isMale
0,829,Chiraphon,False,True
1,428,Benjakalayanee,False,True
2,2259,Phen,False,True
3,1644,Muhamat,True,True
4,32,Aekapon,True,True
...,...,...,...,...
1246,266,Aphiwat,True,True
1247,687,Chali,True,True
1248,2205,Phanumat,True,False
1249,2903,Sasi,False,True


0.5115907274180655

In [ ]:
# dtree = pickle.load(open("model.pkl", "rb"))
# dtree

pred = dtree.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)




array([ True,  True,  True, ...,  True,  True,  True])